In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
BASE_DIR = '../input/tabular-playground-series-apr-2022/'
X_train = pd.read_csv(BASE_DIR + "train.csv")
y_train = pd.read_csv(BASE_DIR + "train_labels.csv")
X_test = pd.read_csv(BASE_DIR + "test.csv")
y_test = pd.read_csv(BASE_DIR + "sample_submission.csv")

In [ ]:
print(X_train.isnull().sum())
X_train.info

In [ ]:
print(X_train.corr())
plt.subplots(figsize=(14,10))
sns.heatmap(X_train.corr())

In [ ]:
X_train = X_train.merge(y_train, on='sequence')
X_test = X_test.merge(y_test, on='sequence')
print(X_train['state'].value_counts())
print(X_train['step'].value_counts().index)
print(X_train.columns)

In [ ]:
c = 0
for i in X_train.columns:
    X_train.boxplot(i)
    plt.show()

In [ ]:
print(pd.pivot_table(X_train, index = 'state', columns = 'sequence', values = 'subject'))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
y_train_reshaped = X_train['state'].copy()
#X_train.drop(['state'], inplace=True, axis=1)
X_train.columns
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)
X_test.drop(['state'], inplace=True, axis=1)
print(X_train.columns)
print(y_train_reshaped)

In [ ]:
from sklearn.model_selection import train_test_split
X_training,X_val,y_training,y_val=train_test_split(X_train_scaled,y_train_reshaped,test_size=0.20, random_state=33)

In [ ]:
X_training = np.delete(X_training,X_training.shape[1]-1,1)
X_test_scaled = np.delete(X_test_scaled,X_test_scaled.shape[1]-1,1)

In [ ]:
X_training.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 33)
dt.fit(X_training, y_training)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 33)
#lr.fit(X_train_scaled, y_train_reshaped)

In [ ]:
from sklearn.model_selection import cross_val_score
cv_lr = cross_val_score(lr,X_training,y_training,cv=5)
print("Accuracy: {:.2f} %".format(cv_lr.mean()*100))
print("Standard Deviation: {:.2f} %".format(cv_lr.std()*100))

In [ ]:
#simple performance reporting function
def clf_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

In [ ]:
from sklearn.model_selection import GridSearchCV 

param_grid = {'max_iter' : [2000],
              'penalty' : ['l1', 'l2'],
              'C' : np.logspace(-4, 4, 20),
              'solver' : ['liblinear']}

clf_lr = GridSearchCV(lr, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_lr = clf_lr.fit(X_training,y_training)
clf_performance(best_clf_lr,'Logistic Regression')

In [ ]:
from lightgbm import LGBMClassifier
LGB = LGBMClassifier(random_state=33)
LGB.fit(X_training, y_training)

In [ ]:
cv_lgb = cross_val_score(LGB,X_training,y_training,cv=5)
print("Accuracy: {:.2f} %".format(cv_lgb.mean()*100))
print("Standard Deviation: {:.2f} %".format(cv_lgb.std()*100))

In [ ]:
inverse = sc.inverse_transform(X_val)
y_val = inverse[:,inverse.shape[1]-1]
print(y_val)
X_val = np.delete(X_val,X_val.shape[1]-1,1)
y_pred_val_dt = dt.predict(X_val)
from sklearn.metrics import confusion_matrix, accuracy_score
print(y_pred_val_dt.shape)
cm = confusion_matrix(y_val, y_pred_val_dt)
print(cm)
accuracy_score(y_val, y_pred_val_dt)

In [ ]:
y_pred_dt = dt.predict(X_test_scaled)
from sklearn.metrics import confusion_matrix, accuracy_score
X_test = X_test.merge(y_test, on='sequence')
y = X_test['state'].copy()
cm = confusion_matrix(y, y_pred_dt)
print(cm)
accuracy_score(y, y_pred_dt)

In [ ]:
y_pred_val = best_clf_lr.predict(X_val)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_val, y_pred_val)
print(cm)
accuracy_score(y_val, y_pred_val)

In [ ]:
y_pred_lr = best_clf_lr.predict(X_test_scaled)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y, y_pred_lr)
print(cm)
accuracy_score(y, y_pred_lr)

In [ ]:
y_pred_val2 = LGB.predict(X_val)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_val, y_pred_val2)
print(cm)
accuracy_score(y_val, y_pred_val2)

In [ ]:
y_pred_lgb = LGB.predict(X_test_scaled)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y, y_pred_lgb)
print(cm)
accuracy_score(y, y_pred_lgb)

In [ ]:
pred = pd.DataFrame({"sequence":X_test['sequence'],
                     "state":y_pred_lgb.tolist()})
pred['state'].value_counts()
pred

In [ ]:
pred_fin = pred.groupby('sequence').max().reset_index()
pred_fin['state'].unique()
pred_fin

In [ ]:
y_test['state'] = pred_fin['state']
y_test

In [ ]:
y_test.to_csv('submission.csv', index=False)